In [37]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [17]:
from time import sleep, time_ns

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import pandas as pd


In [29]:
"""
The code below for chrome browser in linux.
path = "/usr/bin/chromedriver"
option = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=path, options=option)
"""
url = "http://115.124.110.196:8080/epeek/"
browser = webdriver.Firefox()
browser.get(url)
browser.implicitly_wait(2) #wait for two second to load the website fully

#Select the value from the season dropdown
season_drop = Select(browser.find_element(By.ID, 'seasonOpt'))
season_drop.select_by_value('उन्हाळी')
sleep(1)

#Select the value from the division dropdown
div_drop = Select(browser.find_element(By.ID, 'divOpt'))
div_drop.select_by_value('amravati')
sleep(1)

#Select the value from the district dropdown
dist_drop = Select(browser.find_element(By.ID, 'distOpt'))
dist_drop.select_by_value('-')
sleep(1)

#Select the values from the village dropdown
village_drop = Select(browser.find_element(By.ID, 'villOpt'))
village_drop.select_by_value('-')

#Select button
button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/table/tbody/tr[11]/td/button')
button.click()
sleep(20)

object_content = browser.find_element(By.TAG_NAME, 'object')
browser.switch_to.frame(object_content)

rows = len(browser.find_elements(By.CSS_SELECTOR, 'tr'))
print(rows)
row = 2
data_list = []
while(row<rows-1):
    data = {}
    try:
        data['sl_no'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[1]').text
        data['village'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[2]').text
        data['survey_account_number'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[3]').text
        data['peak_monitoring_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[4]').text
        data['permanent_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[5]').text
        data['current_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[6]').text
        data['total_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[7]').text
        
        data_list.append(data)
        row += 1

    except NoSuchElementException:
        break

print(data_list)

data_set = pd.DataFrame(data_list)
data_set.to_csv(f'data/data_{time_ns()}.csv')


browser.close()



8
[{'sl_no': '1', 'village': 'अकोला', 'survey_account_number': '4069', 'peak_monitoring_area': '5956.97', 'permanent_area': '0.00', 'current_area': '29.76', 'total_area': '5986.73'}, {'sl_no': '2', 'village': 'अमरावती', 'survey_account_number': '3616', 'peak_monitoring_area': '5386.93', 'permanent_area': '0.00', 'current_area': '14.59', 'total_area': '5401.52'}, {'sl_no': '3', 'village': 'बुलडाणा', 'survey_account_number': '9050', 'peak_monitoring_area': '12503.17', 'permanent_area': '0.00', 'current_area': '35.94', 'total_area': '12539.11'}, {'sl_no': '4', 'village': 'यवतमाळ', 'survey_account_number': '3968', 'peak_monitoring_area': '6788.61', 'permanent_area': '0.00', 'current_area': '47.67', 'total_area': '6836.28'}, {'sl_no': '5', 'village': 'वाशिम', 'survey_account_number': '1364', 'peak_monitoring_area': '2125.23', 'permanent_area': '0.00', 'current_area': '16.05', 'total_area': '2141.29'}]
